# [Bonus Tasks](svm_segm.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.ndimage as ndi
import skimage.morphology as morph
import skimage.util
from sklearn.pipeline import make_pipeline
from skimage.util import view_as_blocks
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [2]:
def blend_result(img, seg, border_radius=1, interior_opacity=1, interior_opacity_decay=0.9, color=(0,1,0)):
    img  = np.dstack([img] * 3).copy()
    img -= img.min()
    img /= img.max()
    selem  = morph.disk(border_radius)
    seg_bd = np.logical_xor(morph.binary_dilation(seg, selem), morph.binary_erosion(seg, selem))
    mask_decay = ndi.distance_transform_edt(seg)
    for i in range(3):
        opacity = interior_opacity / pow(1 + mask_decay[seg], interior_opacity_decay)
        img[:,:,i][seg] = color[i] * opacity + (1 - opacity) * img[:,:,i][seg]
        img[:,:,i][seg_bd] = color[i]
    return img

In [3]:
patch_size = 32

---

**[Task 1.1.]()** Implement `create_data_matrix`:

In [4]:
def create_data_matrix(img):
    patches = view_as_blocks(img, block_shape=(patch_size, patch_size))
    matrix_resh = patches.reshape(-1, patch_size * patch_size)
    return matrix_resh


**[Task 1.2.]()** Implement `create_gt_labels_vector`:

In [5]:
def create_gt_labels(gt):
    patches_gt = view_as_blocks(gt, block_shape=(patch_size, patch_size))
    a, b = patches_gt.shape[:2]
    vec_reshaped = patches_gt.reshape(a * b, -1)
    fg = vec_reshaped.mean(axis=1)
    labs = np.where(fg > 0.5, 1, np.where(fg == 0, -1, 0))
    return labs

**[Task 1.3.]()** Create the SVM classifier:

In [6]:
clf = make_pipeline(StandardScaler(), SVC(class_weight='balanced', gamma=0.1))

**[Task 1.3 (a).]()** Create the data matrices for the images `dna-33` and `dna-44`:

In [7]:
dna_33 = plt.imread('data/NIH3T3/im/dna-33.png')
dna_44 = plt.imread('data/NIH3T3/im/dna-44.png')

gt_33 = plt.imread('data/NIH3T3/gt/33.png')
gt_44 = plt.imread('data/NIH3T3/gt/44.png')

dm_33 = create_data_matrix(dna_33)
dm_44 = create_data_matrix(dna_44)


**[Task 1.3 (b).]()** Create the corresponding ground truth label vectors:

In [8]:
gtlab_33 = create_gt_labels(gt_33)
gtlab_44 = create_gt_labels(gt_44)

print(gtlab_33, gtlab_44)

[-1 -1 -1 ...  0 -1 -1] [-1 -1 -1 ... -1 -1 -1]


**[Task 1.3 (c).]()** Create the *combined* data matrices and ground truth label vectors:

In [9]:
#filter rows
valid_indices_33 = np.where((gtlab_33 == 1) | (gtlab_33 == -1))[0]
valid_indices_44 = np.where((gtlab_44 == 1) | (gtlab_44 == -1))[0]

dm_33_filtered = dm_33[valid_indices_33]
dm_44_filtered = dm_44[valid_indices_44]

gtlab_33_filtered = gtlab_33[valid_indices_33]
gtlab_44_filtered = gtlab_44[valid_indices_44]

# Stack the filtered data matrices and label vectors
data_matrix = np.concatenate((dm_33_filtered, dm_44_filtered), axis=0)
label_vector = np.concatenate((gtlab_33_filtered, gtlab_44_filtered), axis=0)

# Print to verify
print("Data Matrix Shape:", data_matrix.shape)
print("Label Vector Shape:", label_vector.shape)

Data Matrix Shape: (2109, 1024)
Label Vector Shape: (2109,)


**[Task 1.3 (d).]()** Train the classifier using the data matrix and label vectors from above:

In [10]:
clf.fit(data_matrix, label_vector)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(class_weight='balanced', gamma=0.1))])

**[Task 1.4.]()** Implement the function `predict_image`:

In [11]:
def predict_image(img):
    dm_img = create_data_matrix(img)
    gtlab_img = create_gt_labels(img)
    valid_indices_img = np.where((gtlab_img == 1) | (gtlab_img == -1))[0]
    dm_img_filtered = dm_img[valid_indices_img]
    gtlab_img_filtered = gtlab_img[valid_indices_img]
    clf = SVC()
    clfit = clf.fit(dm_img_filtered, gtlab_img_filtered)
    predicted_labels = clf.predict(dm_img)
    seg = predicted_labels.reshape((img.shape[0] // patch_size, img.shape[1] // patch_size))
    return seg

    

    

In [13]:
def predict_image(img):
    # Create data matrix and ground truth labels for the image
    dm_img = create_data_matrix(img)
    gtlab_img = create_gt_labels(img)
    
    # Filter valid indices where label is -1 or +1 (foreground/background)
    valid_indices_img = np.where((gtlab_img == 1) | (gtlab_img == -1))[0]
    
    # Debug: Print the number of valid patches
    print(f"Valid patches for training: {len(valid_indices_img)}")
    
    # If no valid patches are found, return an error or skip
    if len(valid_indices_img) == 0:
        print("No valid foreground or background patches found. Skipping training.")
        return None  # Or return a default segmentation mask if needed
    
    # Filter the data matrix and labels using the valid indices
    dm_img_filtered = dm_img[valid_indices_img]
    gtlab_img_filtered = gtlab_img[valid_indices_img]
    
    # Initialize and train the classifier
    clf = SVC()  # Or use your classifier of choice
    clf.fit(dm_img_filtered, gtlab_img_filtered)
    
    # Predict segmentation labels for the entire image
    predicted_labels = clf.predict(dm_img)  # Predict on the whole data matrix of the image
    
    # Reshape predicted labels to match the shape of the image patches
    seg = predicted_labels.reshape((img.shape[0] // patch_size, img.shape[1] // patch_size))
    
    return seg

# Load image and predict segmentation
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

if seg is not None:
    # Visualize the segmentation result
    plt.figure()
    plt.imshow(blend_result(img, seg))
else:
    print("Segmentation not performed due to insufficient valid data.")


Valid patches for training: 0
No valid foreground or background patches found. Skipping training.
Segmentation not performed due to insufficient valid data.


Test your implementation:

In [ ]:
img = plt.imread(f'data/NIH3T3/im/dna-0.png')
seg = predict_image(img)

plt.figure()
plt.imshow(blend_result(img, seg))

TypeError: blend_result() missing 1 required positional argument: 'seg'

<Figure size 640x480 with 0 Axes>

**[Task 1.5.]()** Perform batch processing: